# Local-Citation-Recommendation
Code for ECIR 2022 paper [Local Citation Recommendation with Hierarchical-Attention Text Encoder and SciBERT-based Reranking](https://link.springer.com/chapter/10.1007/978-3-030-99736-6_19)

# Update 07-11-2022
1. Cleaned the code for training and testing the prefetching system, to make it easier to read and to run.
2. Simplify the information in config file, now there is only one global configuration file for prefetching and it is more readable.
3. Optimize the GPU usage, now the system can be trained using a single GPU.
4. Introduced the structure of the dataset and showed how to build your custom dataset and train a citation recommendation system on that.
5. Provided a step-by-step tutorial on google colab, illustrating the whole process of training and testing of the entire prefetching and reranking system.

# Hardware Requirement
1. OS: Ubuntu 20.04 or 18.04
2. 1 or more GPUs

# Install Dependencies

In [ ]:
!rm -r *
!git clone https://github.com/nianlonggu/Local-Citation-Recommendation.git

!pip install numpy tqdm matplotlib nltk transformers -q
!pip install gdown -q
!pip install cupy-cuda11x  ## suppose CUDA version >= 11.2
import torch
torch.__version__
import nltk
nltk.download('omw-1.4', quiet=True)
nltk.download('stopwords',quiet=True)
nltk.download('wordnet',quiet=True)
import os
os.chdir("Local-Citation-Recommendation")

# Download Glove Embedding 
For simplicity, we refer **MAIN** as the main folder of the repo.

In [ ]:
!gdown  https://drive.google.com/uc?id=1T2R1H8UstSILH_JprUaPNY0fasxD2Txr; unzip model.zip; rm model.zip

# Prepare Dataset

## Option 1: Build your custom dataset 
**This github repo contains a "pseudo" custom dataset that is actually ACL-200**

The custom dataset will contain 5 components: contexts, papers, training/validation/test sets

In [ ]:
import json
contexts = json.load(open("data/custom/contexts.json"))
papers = json.load(open("data/custom/papers.json"))
train_set = json.load(open("data/custom/train.json"))
val_set = json.load(open("data/custom/val.json"))
test_set = json.load(open("data/custom/test.json"))

### contexts contain the local contexts that cite a paper.

In [ ]:
for key in contexts:
    break
contexts[key]

### papers contain the papers database, each paper has title and abstract.


In [ ]:
for key in papers:
    break
papers[key]

### train/val/test set contain the context_id (used for get the local context information and cited and citing papers information)

In [ ]:
train_set[0]

positive_ids means the paper that is actually cited by the context. In this experiment the positive_ids always has one paper.

You can create you own dataset with the same structure, and then train the citation recommendation system.

## Option 2: Download Processed Dataset
You can also download our **processed dataset** (and **pretrained models**) from [Google Drive](https://drive.google.com/drive/folders/1QwQuJsBOGEESFTgl-7wWbqcig7vJNlQ2?usp=sharing)

(There can be some additional information in the processed dataset other than what have been displayed in the examples above. They are irrelevant information.)

# Prefetching Part
In the following experiment, we use the "custom" dataset as an example. This dataset is the same as the ACL dataset. If you have created you dataset, you need to modify the config file at

MAIN/src.prefetch/config/YOUR_DATASET_NAME/global_config.cfg
## Training

This can take around 2h for this custom dataset (the same as ACL-200)

In [ ]:
!cd src/prefetch; python run.py -config_file_path config/custom/global_config.cfg -mode train

This code will automatically handle the loop of  training -> updating paper embeddings -> updating prefetched candidates for contructing triplets -> resuming training.


In this case, the model checkpoint will be stored in the folder "MAIN/model/prefetch/custom/"; <br>
              the paper embeddings are stored in "MAIN/embedding/prefetch/custom/"; <br>
              the training log files are stored in "MAIN/log/prefetch/custom/"; 
              
The file MAIN/log/prefetch/custom/validate_NN.log contains the validation performance of each checkpoint during training. With this information, we can pick up the best-performance model for testing. 

You can specify where to store the checkpoint, log files and other parammeters by modifying the config/custom/global_config.cfg configuration file.

Note: **Before testing, after determining the best checkpoint, removing all the other checkpoints. If there are multiple checkpoints in MAIN/model/prefetch/custom/, the model will use the latest checkpoint by default.**

## Testing
To test the performance of the prefetching model, we need to first use the model checkpoint to compute the embedding for each paper in the database. This paper embedding is the index of the paper database, which is then used to perform nearest neighbor search. Then the next step is the test the prefetching performance.


Here I download the pretrained ACL model only for demonstration. If you are training the model using your custom data, you can wait until the training ends and select the best checkpoint based on the validation performance.

In [ ]:
try:
    os.system("rm -r model/prefetch/custom")
    os.makedirs("model/prefetch/custom")
except:
    pass
!cd model/prefetch/custom; gdown  https://drive.google.com/uc?id=13J5mtRg6t3Lcsn6fCdLJ1pZhtXnpREEq;

Step 1: Compute the embeddings of all papers in the database, using the trained text encoder (HAtten).

In [ ]:
!cd src/prefetch; python run.py -config_file_path config/custom/global_config.cfg -mode compute_paper_embedding

Step 2: Test the prefetching performance on the test set

In [ ]:
!cd src/prefetch; python run.py -config_file_path config/custom/global_config.cfg -mode test

# Reranking Part

## Training
In order to train the reranker, we need to first create the training dataset. More specifically, for each query in the training set, we first use the trained HAtten prefetcher to prefetch a list of (2000) candidates. Then within the 2000 prefetched candidates we can construct triplets to train the reranker.


At this stage, we should have trained the prefetching model. We need to 1) compute paper embeddings for prefetching; 2) use the prefetching model to obtain prefetched candidates for each training example; 3) use the training examples with prefetched candidates to fine-tune SciBERT reranker. 

In [ ]:
!cd src/prefetch; python run.py -config_file_path config/custom/global_config.cfg -mode compute_paper_embedding

In [ ]:
!cd src/prefetch; python run.py -config_file_path config/custom/global_config.cfg -mode get_training_examples_with_prefetched_ids_for_reranking

In [ ]:
!cd src/prefetch; python run.py -config_file_path config/custom/global_config.cfg -mode get_val_examples_with_prefetched_ids_for_reranking

After get the prefetched ids for training and validstion set, we can start training the reranker: (This can take 2.5 h to finish one epoch on this custom dataset)

In [ ]:
!cd src/rerank; python train.py -config_file_path  config/custom/scibert/training_NN_prefetch.config

# Use The HAtten Prefetcher and the SciBERT Reranker in Python Code

Before run this code, we should have trained the prefetching and the reranking models and know the path to the checkpoint of the saved model. (Here I download the pretrained model on ACL-200 for demonstration. If you train using your custom data, skip this and put the trained models' checkpoint to the corresponding model folder.)

In [ ]:
try:
    os.system("rm -r model/prefetch/custom")
    os.makedirs("model/prefetch/custom")
except:
    pass
!cd model/prefetch/custom; gdown  https://drive.google.com/uc?id=13J5mtRg6t3Lcsn6fCdLJ1pZhtXnpREEq;

try:
    os.system("rm -r model/rerank/custom")
    os.makedirs("model/rerank/custom/scibert/NN_prefetch")
except:
    pass
!cd model/rerank/custom/scibert/NN_prefetch; gdown  https://drive.google.com/uc?id=1DmSw6HR2W4fbUKp24K1_TREPhlLiQuEb;

## Load models

In [ ]:
from citation_recommender import * 
prefetcher = Prefetcher( 
       model_path="model/prefetch/custom/model_batch_35000.pt",
       embedding_path="embedding/prefetch/custom/paper_embedding.pkl", ## make sure the papers embeddings have been computed
       gpu_list= [0,] 
)
reranker = Reranker( model_path = "model/rerank/custom/scibert/NN_prefetch/model_batch_91170.pt", 
                     gpu_list = [0,] )

## Get paper recommendations

Then we can construct a query, and use the query to find n most relevant papers. The query is a dictionary containing 3 keys: "citing_title","citing_abstract" and "local_context". We can get some real example from the test set, or we can contruct a simple query as follows:

(## You can specify any other values, e.g., 100, 1000 or 2000. Note that the reranking time is proportional to the number of candidates to rerank.) 

In [ ]:
idx = 100
context_info = contexts[test_set[idx]["context_id"]]
citing_id = context_info["citing_id"]
refid = context_info["refid"]  ## The ground-truth cited paper

local_context = context_info["masked_text"]
citing_paper = papers[citing_id]
citing_title = citing_paper["title"]
citing_abstract = citing_paper["abstract"]

In [ ]:
citing_title, citing_abstract, local_context, refid

In [ ]:
candi_list = prefetcher.get_top_n(
  {
      "citing_title":citing_title,
      "citing_abstract":citing_abstract,
      "local_context":local_context
  }, 500
)
print(candi_list[:10])

for pos, cadi_id in enumerate(candi_list):
    if cadi_id == refid:
        print("The truely cited paper's id %s appears in position: %d among the prefetched ids."%( refid, pos ))
        break
if refid not in candi_list:
    print("The truely cited paper's id %s is not included in the prefetched ids"%( refid ))

Then we can rerank the prefetched candidates

In [ ]:
candidate_list = [  {"paper_id": pid,
                     "title":papers[pid].get("title",""),
                     "abstract":papers[pid].get("abstract","")}
                            for pid in candi_list ] 
# start reranking
reranked_candidate_list = reranker.rerank( citing_title,citing_abstract,local_context, candidate_list )
reranked_candidate_ids = [item["paper_id"] for item in reranked_candidate_list]

In [ ]:
for pos, cadi_id in enumerate(reranked_candidate_ids):
    if cadi_id == refid:
        print("The truely cited paper's id %s appears in position: %d among the reranked ids."%( refid, pos ))
        break
if refid not in reranked_candidate_ids:
    print("The truely cited paper's id %s is not included in the reranked ids"%( refid ))

## Evaluation of the whole prefetching and reranking pipeline


We use HAtten to prefetch 100 candidates and then rerank them and we record the Recall@10 in the final recommendations  (We test this on 100 test examples only for demonstration)

In [ ]:
hit_list = []
top_K = 10

for idx in tqdm(range(100)):

    context_info = contexts[test_set[idx]["context_id"]]
    citing_id = context_info["citing_id"]
    refid = context_info["refid"]  ## The ground-truth cited paper

    local_context = context_info["masked_text"]
    citing_paper = papers[citing_id]
    citing_title = citing_paper["title"]
    citing_abstract = citing_paper["abstract"]

    candi_list = prefetcher.get_top_n(
        {
            "citing_title":citing_title,
            "citing_abstract":citing_abstract,
            "local_context":local_context
        }, 100  ## 100 candidates 
    )

    candidate_list = [  {"paper_id": pid,
                     "title":papers[pid].get("title",""),
                     "abstract":papers[pid].get("abstract","")}
                            for pid in candi_list ] 
    # start reranking
    reranked_candidate_list = reranker.rerank( citing_title,citing_abstract,local_context, candidate_list )
    reranked_candidate_ids = [item["paper_id"] for item in reranked_candidate_list]

    hit_list.append( refid in reranked_candidate_ids[:top_K])

In [ ]:
print("The average recall@%d: %.4f"%( top_K, np.mean(hit_list)))

This value is close to the results on ACL-200 in Table 4 in the paper, where we tested using full test set.

# References
When using our code or models for your application, please cite the following paper:

```
@InProceedings{10.1007/978-3-030-99736-6_19,
author="Gu, Nianlong
and Gao, Yingqiang
and Hahnloser, Richard H. R.",
editor="Hagen, Matthias
and Verberne, Suzan
and Macdonald, Craig
and Seifert, Christin
and Balog, Krisztian
and N{\o}rv{\aa}g, Kjetil
and Setty, Vinay",
title="Local Citation Recommendation with Hierarchical-Attention Text Encoder and SciBERT-Based Reranking",
booktitle="Advances in Information Retrieval",
year="2022",
publisher="Springer International Publishing",
address="Cham",
pages="274--288",
abstract="The goal of local citation recommendation is to recommend a missing reference from the local citation context and optionally also from the global context. To balance the tradeoff between speed and accuracy of citation recommendation in the context of a large-scale paper database, a viable approach is to first prefetch a limited number of relevant documents using efficient ranking methods and then to perform a fine-grained reranking using more sophisticated models. In that vein, BM25 has been found to be a tough-to-beat approach to prefetching, which is why recent work has focused mainly on the reranking step. Even so, we explore prefetching with nearest neighbor search among text embeddings constructed by a hierarchical attention network. When coupled with a SciBERT reranker fine-tuned on local citation recommendation tasks, our hierarchical Attention encoder (HAtten) achieves high prefetch recall for a given number of candidates to be reranked. Consequently, our reranker requires fewer prefetch candidates to rerank, yet still achieves state-of-the-art performance on various local citation recommendation datasets such as ACL-200, FullTextPeerRead, RefSeer, and arXiv.",
isbn="978-3-030-99736-6"
}
```